In [17]:
from rag_module.rag import explainaRAG
import mlflow
import inspect
import pandas as pd
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# set path to benchmark data
os.environ["embedding_path"] = "./embeddings/ELOQ_news/"

In [44]:
rags = [{"model": explainaRAG(normalize_embeddings=True), "name": "normalized"},
        {"model": explainaRAG(normalize_embeddings=False), "name": "regular"}]

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Loading default flashrank model for language en
Default Model: ms-marco-MiniLM-L-12-v2
Loading FlashRankRanker model ms-marco-MiniLM-L-12-v2 (this message can be suppressed by setting verbose=0)
Loading model FlashRank model ms-marco-MiniLM-L-12-v2...


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Loading default flashrank model for language en
Default Model: ms-marco-MiniLM-L-12-v2
Loading FlashRankRanker model ms-marco-MiniLM-L-12-v2 (this message can be suppressed by setting verbose=0)
Loading model FlashRank model ms-marco-MiniLM-L-12-v2...


In [ ]:
chunkdata = pd.read_json(rags[0]['model'].embedder.chunkdata_path)
evalset = pd.read_csv("eval_data/ScopeQA/ELOQ_silver.csv")
evalset = evalset.merge(chunkdata, how="left", left_on="doc_id", right_on="filename").drop(columns=["filename", "text"])

In [65]:
sampleset = evalset.sample(200)

In [66]:
def retrieve_docs(rag, question):
    result = rag['model'].retrieve(question)
    return [doc['vector_id'] for doc in result]

for rag in rags:
    sampleset[f'{rag['name']}_docs'] = sampleset['question'].apply(lambda q: retrieve_docs(rag, q))

Batches: 100%|██████████| 1/1 [00:00<00:00, 49.74it/s]


In [75]:
(sampleset['normalized_docs'] == sampleset['regular_docs']).value_counts()

True    200
Name: count, dtype: int64